In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Desnutrición por departamento**

####**Descripción:**
Genera una tabla de datos con la cantidad de beneficiarios total y en cada condición por departamento, y per cápita, según datos poblacionales del Censo Nacional de Población y Vivienda 2018. Además calcula:
- de los beneficiarios en cada departamento, la fracción que sufrió desnutrición al menos una vez;
- de los beneficiarios que sufrieron desnutrición, la fracción que se recuperó;
- de los beneficiarios que se recuperaron, la fracción que reincidió en desnutrición.
 
####**Requerimientos:**

- Tabla de datos `clean_data/sociodemo.parquet` con los datos sociodemográficos y las etiquetas de cada beneficiario.
- Tabla de relación entre los identificadores IdBeneficiario y las etiquetas `clean_data/IdBeneficiario_labels.parquet`.
- Tabla de datos `auxiliary_data/municipio_poblacion_sexo_edad_pib.csv` con datos demográficos y económicos por municipio procesado a partir de la [*Serie municipal de población por área, sexo y edad, para el periodo 2018 -2026*](https://www.dane.gov.co/index.php/estadisticas-por-tema/demografia-y-poblacion/proyecciones-de-poblacion) con las proyecciones poblacionales elaboradas por el DANE a partir de los datos del Censo Nacional de Población y Vivienda 2018. Contiene la población proyectada de niños y niñas de 0 a 5 años, discriminada por sexo, edad y año y el PIB para cada municipio.
 
####**Salidas:**

- Tabla de datos `app/incidencia_dpto.parquet` con la cantidad de beneficiarios en cada condición y las fracciones relevantes por departamento.

In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np

### **Lectura de archivos**

In [ ]:
t0 = time.time()
# Lee el archivo con datos sociodemográficos 
soc = pd.read_parquet('clean_data/sociodemo.parquet')
# Lee el archivo con las etiquetas
labels = pd.read_parquet('clean_data/IdBeneficiario_labels.parquet')
# Asigna a cada beneficiario la etiqueta correspondiente
soc = soc.merge(labels, on='IdBeneficiario', how='left')
# Lee el archivo con datos demográficos por municipio y departamento
geo = pd.read_csv('auxiliary_data/municipio_poblacion_sexo_edad_pib.csv')
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2244 entries, 0 to 2243
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2244 non-null   int64  
 1   DP             2244 non-null   int64  
 2   DPNOM          2244 non-null   object 
 3   DPMP           2244 non-null   int64  
 4   MPIO           2244 non-null   object 
 5   AÑO            2244 non-null   int64  
 6   Hombres_0      2244 non-null   int64  
 7   Hombres_1      2244 non-null   int64  
 8   Hombres_2      2244 non-null   int64  
 9   Hombres_3      2244 non-null   int64  
 10  Hombres_4      2244 non-null   int64  
 11  Hombres_5      2244 non-null   int64  
 12  Mujeres_0      2244 non-null   int64  
 13  Mujeres_1      2244 non-null   int64  
 14  Mujeres_2      2244 non-null   int64  
 15  Mujeres_3      2244 non-null   int64  
 16  Mujeres_4      2244 non-null   int64  
 17  Mujeres_5      2244 non-null   int64  
 18  Total Ho

### **Exploración**

In [ ]:
# Número de municipios en nuestro dataset vs el total de municipios en Colombia
soc.cod_mpio.nunique(), geo.DPMP.nunique()

(1050, 1122)

In [ ]:
# Tabla de datos con proyecciones poblacionales por municipio, sexo y edad
geo.head()

Unnamed: 0  DP      DPNOM  ...  Total Mujeres    Total           PIB
0           0   5  Antioquia  ...        1286471  2427129  55082.301180
1           1   5  Antioquia  ...        1314619  2483545  59974.806184
2           2   5  Antioquia  ...           9833    20367    231.172055
3           3   5  Antioquia  ...           9748    20258    257.196344
4           4   5  Antioquia  ...           1243     2695     28.562662

[5 rows x 22 columns]

In [ ]:
# Tabla de relación entre códigos y nombres de departamentos
print(geo[['DP', 'DPNOM']].drop_duplicates().sort_values('DP'))

      DP                       DPNOM
0      5                   Antioquia
250    8                   Atlántico
296   11                Bogotá, D.C.
298   13                     Bolívar
390   15                      Boyacá
636   17                      Caldas
690   18                     Caquetá
722   19                       Cauca
806   20                       Cesar
856   23                     Córdoba
916   25                Cundinamarca
1148  27                       Chocó
1208  41                       Huila
1282  44                  La Guajira
1312  47                   Magdalena
1372  50                        Meta
1430  52                      Nariño
1558  54          Norte de Santander
1638  63                     Quindio
1662  66                   Risaralda
1690  68                   Santander
1864  70                       Sucre
1916  73                      Tolima
2010  76             Valle del Cauca
2094  81                      Arauca
2108  85                    Casanare
2

### **Población en cada condición por departamento**

In [ ]:
# Cuenta los beneficiarios por departamento, en total y en cada condición
# (desnutrición, recuperación, reincidencia)
dpto_incidencia = soc.groupby('cod_dpto')[['IdBeneficiario', 'desnutricion', 'recuperacion', 'reincidencia']].\
  apply(lambda x: (x > 0).sum()).reset_index().rename(columns={'IdBeneficiario': 'n_beneficiarios'})
dpto_incidencia.head()

cod_dpto  n_beneficiarios  desnutricion  recuperacion  reincidencia
0         5           189681          9104          7768           778
1         8            53269          2341          2094           200
2        11           171851          7270          5853           481
3        13            72883          5076          4379           554
4        15            42473          2677          2372           216

In [ ]:
# Toma cada columna y la divide por la anterior para hallar:
# - de los beneficiarios en cada departamento, la fracción que sufrió desnutrición al menos una vez
# - de los beneficiarios que sufrieron desnutrición, la fracción que se recuperó
# - de los beneficiarios que se recuperaron, la fracción que reincidió
for col_index in range(3):
  col_num = dpto_incidencia.columns[col_index + 2]
  col_denom = dpto_incidencia.columns[col_index + 1]
  dpto_incidencia[f'fracc_{col_num}'] = dpto_incidencia[col_num] / dpto_incidencia[col_denom]
dpto_incidencia.head()

cod_dpto  n_beneficiarios  ...  fracc_recuperacion  fracc_reincidencia
0         5           189681  ...            0.853251            0.100154
1         8            53269  ...            0.894490            0.095511
2        11           171851  ...            0.805089            0.082180
3        13            72883  ...            0.862687            0.126513
4        15            42473  ...            0.886066            0.091062

[5 rows x 8 columns]

### **Cruce con los datos del Censo Nacional de Población y Vivienda 2018**

Se utiliza la población de 2018 como el estimado más general de la población por departamento, pero se tienen en cuenta los beneficiarios atendidos en los tres años (2017 a 2019).

In [ ]:
# Población y PIB por departamento
geo2018 = geo.loc[geo['AÑO'] == 2018, ['DP', 'DPNOM', 'Total', 'PIB']]
geo2018_dpto = geo2018.groupby(['DP', 'DPNOM'])['Total', 'PIB'].sum().reset_index()
geo2018_dpto.columns = ['cod_dpto', 'dpto', 'poblacion', 'PIB']
geo2018_dpto.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


cod_dpto          dpto  poblacion            PIB
0         5     Antioquia    6407102  128910.366015
1         8     Atlántico    2535517   38947.399571
2        11  Bogotá, D.C.    7412566  227062.690663
3        13       Bolívar    2070110   31485.047361
4        15        Boyacá    1217376   24885.323042

In [ ]:
# Cruce de tablas
join = geo2018_dpto.merge(dpto_incidencia, on='cod_dpto', how='inner')
join.head(2)

cod_dpto       dpto  ...  fracc_recuperacion  fracc_reincidencia
0         5  Antioquia  ...            0.853251            0.100154
1         8  Atlántico  ...            0.894490            0.095511

[2 rows x 11 columns]

In [ ]:
# Poblaciones en cada condición (en nuestros datos) y del PIB per cápita (por cada 100 000 individuos)
for col in ['desnutricion', 'recuperacion', 'reincidencia', 'PIB']:
  join[f'{col}_per_cap'] = join[col] / join['poblacion'] * 1e5
join.head()

cod_dpto          dpto  ...  reincidencia_per_cap  PIB_per_cap
0         5     Antioquia  ...             12.142775  2011.991787
1         8     Atlántico  ...              7.887938  1536.073297
2        11  Bogotá, D.C.  ...              6.488981  3063.213072
3        13       Bolívar  ...             26.761863  1520.935958
4        15        Boyacá  ...             17.743080  2044.177234

[5 rows x 15 columns]

In [ ]:
# Departamentos con mayor porcentaje de reincidencia en desnutrición entre los beneficiarios recuperados
join.sort_values('fracc_reincidencia', ascending=False).head()

cod_dpto       dpto  ...  reincidencia_per_cap  PIB_per_cap
28        91   Amazonas  ...             26.113411   943.874974
19        66  Risaralda  ...              7.525962  1528.844505
5         17     Caldas  ...             17.029717  1444.628694
31        97     Vaupés  ...             36.767409   667.704279
11        27      Chocó  ...             53.475336   682.957809

[5 rows x 15 columns]

### **Guardado**

In [ ]:
# Guarda la tabla de datos en la carpeta de la aplicación
join.to_parquet('app/incidencia_dpto.parquet')
print(f'Archivo guardado: {time.time() - t0:.2f} s.')

Archivo guardado: 9.10 s.
